# Customer Segmentation with the Tindie Orders' dataset

Customer segmentation will be applied to the KASSER SYNTHS customer database in Tindie using K-means clustering from scikit-learn.

Case Study:
Can this customer database be grouped to develop customized relationships?

To answer this question 3 features will be created and used:
- products ordered (Quantity)
- average return rate (Status Refund / Billed + Refund)
- total spending (Item Total)

Dataset represents real customers & orders data between November 2018 - May 2022 and it is pseudonymized for confidentiality.

Imports

In [1]:
# data wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for data preprocessing and clustering
from sklearn.cluster import KMeans

%matplotlib inline
# to include graphs inline within the frontends next to code

%config InlineBackend.figure_format='retina'
#to enable retina (high resolution) plots

pd.options.mode.chained_assignment = None
# to bypass warnings in various dataframe assignments

Investigate data

In [2]:
# load data into a dataframe
customers_orders = pd.read_csv("https://raw.githubusercontent.com/abcasas/kasser_tindie_stats/main/datasets/orders/orders.csv")

In [3]:
# first rows of the dataset
customers_orders.head()

,Order ID,Order Date,Shipped,Customer ID,City,State/Province,Postal/Zip Code,Country,Product Title,Option Summary,Quantity,Unit Price,Discount Price,Item Total,Shipping Total,Discount Total,Order Total,Status
0,134029,2018-12-07,2018-12-12,79b517750071a0fce0ea0c2ef27fc40d5063df78aac79c...,Brookings,SD,57006,United States of America,DAFM synth - GENESIS YM2612 / YM3438,FM YAMAHA chip: YM3438 - Fully Assembled,1,124.38,124.38,124.38,0.00,0.00,124.38,billed
1,136661,2019-01-04,2019-01-13,5f54c081a80b3cd0960794be1ea8f4fbd1bb977f7d9b30...,Neustadt,RP,67433,Germany,DAFM synth - GENESIS YM2612 / YM3438,FM YAMAHA chip: YM2612 - Fully Assembled,1,126.12,126.12,126.12,0.00,0.00,126.12,billed
2,136829,2019-01-05,2019-01-16,1eaa433ace9b356d976ae83bdfac56282ef0fa7afcc05f...,Auckland,Auckland,1024,New Zealand,DAFM synth - GENESIS YM2612 / YM3438,FM YAMAHA chip: YM2612 - Fully Assembled,1,126.12,126.12,126.12,0.00,0.00,126.12,billed
3,137381,2019-01-10,2019-01-19,92ce259747bc2c850787c5e27547416aa4da0a13a23708...,Berlin,Berlin,10409,Germany,DAFM synth - GENESIS YM2612 / YM3438,FM YAMAHA chip: YM2612 - Fully Assembled,1,126.12,126.12,126.12,0.00,0.00,126.12,billed
4,142040,2019-02-23,2019-02-25,d1b71ad194e919d69cabdf143cea070293c4190d6c3be3...,Bluff City,TN,37618,United States of America,DAFM synth - GENESIS YM2612 / YM3438,FM YAMAHA chip: YM2612 - Fully Assembled,1,156.11,129.99,129.99,19.99,26.12,149.98,billed


In [4]:
# first glance of customers_orders data
customers_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order ID         264 non-null    int64  
 1   Order Date       264 non-null    object 
 2   Shipped          253 non-null    object 
 3   Customer ID      264 non-null    object 
 4   City             264 non-null    object 
 5   State/Province   260 non-null    object 
 6   Postal/Zip Code  264 non-null    object 
 7   Country          264 non-null    object 
 8   Product Title    264 non-null    object 
 9   Option Summary   264 non-null    object 
 10  Quantity         264 non-null    int64  
 11  Unit Price       264 non-null    float64
 12  Discount Price   264 non-null    float64
 13  Item Total       264 non-null    float64
 14  Shipping Total   264 non-null    float64
 15  Discount Total   264 non-null    float64
 16  Order Total      264 non-null    float64
 17  Status          

In [19]:
# descriptive statistics of the non-object columns
customers_orders.describe()

,Order ID,Quantity,Unit Price,Discount Price,Item Total,Shipping Total,Discount Total,Order Total
count,264.000000,264.0,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000
mean,217737.806818,1.0,188.928939,188.063939,188.063939,20.549508,0.865000,210.980682
std,52199.152930,0.0,63.501703,63.478458,63.478458,8.512797,4.563382,69.005841
min,134029.000000,1.0,14.980000,14.980000,14.980000,0.000000,0.000000,14.980000
25%,169716.250000,1.0,156.110000,154.370000,154.370000,16.490000,0.000000,175.820000
50%,213764.500000,1.0,174.990000,174.990000,174.990000,24.990000,0.000000,199.980000
75%,257184.750000,1.0,249.990000,249.990000,249.990000,24.990000,0.000000,274.980000
max,332621.000000,1.0,274.980000,274.980000,274.980000,90.000000,30.000000,374.970000


# 0. Cleaning the data

In [ ]:
customer_orders = customer_orders[['Order ID', 'Order Date', 'Shipped', 'First Name', 'Last Name', 'City','State/Province', 'Postal/Zip Code', 'Country', 'Product Title', 'Option Summary', 'Quantity', 'Unit Price', 'Discount Price', 'Item Total', 'Shipping Total', 'Discount Total', 'Order Total', 'Status' ]]
customer_orders.info()

In [ ]:
orders[orders['Order ID'].duplicated(keep=False)]

In [ ]:
columns_tofill = ['Shipped', 'First Name', 'Last Name', 'City', 'State/Province', 'Postal/Zip Code', 'Country', 'Shipping Total', 'Discount Total', 'Order Total']

for column in columns_tofill:
    orders[column] = orders.groupby('Order ID')[column].transform(lambda x: x.ffill())

orders.info()

In [ ]:
orders[orders['State/Province'].isnull()]

# 1. Products ordered

It is the count of the products ordered in Quantity column by a customer

Create functions to identify customers who order multiple products

In [39]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):
    '''this function:
    1. aggregates a given dataframe by column list, 
    as a result creates a aggregated dataframe by counting the ordered item quantities

    2. adds number_of_X ordered where X is the second element in the column_list 
    to the aggregated dataframe by encoding ordered items into 1

    3. creates final dataframe containing information about 
    how many of X are ordered, based on the first element passed in the column list'''

    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [38]:
# apply functions to customers_orders
customers = aggregate_by_ordered_quantity(customers_orders, ["Customer ID", "Quantity"])

ValueError: cannot insert Quantity, already exists

In [ ]:
print(customers.head())